In [27]:
from arc_circ_sim import ArcCircSim
from utils.get_backend import get_backend

from qiskit import IBMQ, transpile




In [28]:
n = 3
error_list = ArcCircSim.generate_all_unique_pauli_errors(n)
backend = get_backend()

basic_tests = [
   ["XII", "III", "III"],
   ["ZII", "III", "III"],
   ["IXI", "III", "III"],
   ["IZI", "III", "III"],
   ["IIX", "III", "III"],
   ["IIZ", "III", "III"], 
]


experiments = []
for noise in basic_tests:
    res = [0] * 6 # [[arc, trans, job, counts, UFSyndrome, UFResults]]
    res[0] = ArcCircSim(n, pauli_noise_list=noise)
    res[1] = transpile(res[0].code_circ, backend=backend)
    res[2] = backend.run(res[1])
    res[3] = res[2].result().get_counts()
    experiments.append(res) 


for res in experiments:
    print(res[0].pauli_noise_list)
    print(res[3])
    print()


['XII', 'III', 'III']
{'00': 1024}

['ZII', 'III', 'III']
{'10': 1024}

['IXI', 'III', 'III']
{'11': 1024}

['IZI', 'III', 'III']
{'00': 1024}

['IIX', 'III', 'III']
{'00': 1024}

['IIZ', 'III', 'III']
{'01': 1024}



In [29]:
checkm = ArcCircSim.generate_check_matrix(n)
filepath = ArcCircSim.save_external_check_matrix("arccirc_3", checkm)

In [30]:
from mqt.qecc import Code, UFHeuristic
code = Code(filepath)
decoder = UFHeuristic()
decoder.set_code(code)


In [31]:
for res in experiments:
    output = list(res[3].keys())[0]
    uf_format = [True if k == "1" else False for k in output]
    res[4] = uf_format
    decoder.decode(uf_format)
    result = decoder.result
    res[5] = result



In [33]:
for res in experiments:
    print()
    print(res[0].pauli_noise_list)
    print(res[3])
    print(res[4])
    print(res[5])
    


['XII', 'III', 'III']
{'00': 1024}
[False, False]
{
  "decodingTime(ms)": 0,
  "estimate": "[0,0,0]"
}

['ZII', 'III', 'III']
{'10': 1024}
[True, False]
{
  "decodingTime(ms)": 0,
  "estimate": "[0,0,0]"
}

['IXI', 'III', 'III']
{'11': 1024}
[True, True]
{
  "decodingTime(ms)": 0,
  "estimate": "[0,0,0]"
}

['IZI', 'III', 'III']
{'00': 1024}
[False, False]
{
  "decodingTime(ms)": 0,
  "estimate": "[0,0,0]"
}

['IIX', 'III', 'III']
{'00': 1024}
[False, False]
{
  "decodingTime(ms)": 0,
  "estimate": "[0,0,0]"
}

['IIZ', 'III', 'III']
{'01': 1024}
[False, True]
{
  "decodingTime(ms)": 0,
  "estimate": "[0,0,0]"
}
